# ReactAgent

In [1]:
from typing_extensions import override
from vinagent.agent.agent import Agent
import json

class ReactAgent(Agent):
    @override
    def prompt_template(
        self, query: str, user_id: str = "unknown_user", *args, **kwargs
    ) -> str:
        try:
            tools = json.loads(self.tools_path.read_text(encoding="utf-8"))
        except json.JSONDecodeError:
            tools = {}
            self.tools_path.write_text(json.dumps({}, indent=4), encoding="utf-8")

        memory = ""
        if self.memory:
            memory_content = self.memory.load_memory_by_user(
                load_type="string", user_id=user_id
            )
            if memory_content:
                memory = f"- Memory: {memory_content}\n"
        else:
            memory = "- Memory: None\n"
        tools = self.tools_manager.load_tools()
        tool_names = list(tools.keys())
        prompt = (
            "Answer the following questions as best you can. You have access to the following tools:\n"
            f"{tools}\n"
            "Let's consider to memory and user_id:\n"
            f"{memory}\n"
            f"- user_id: {user_id}\n"

            "Use the following format:\n"

            "Question: the input question you must answer\n"
            "Thought: you should always think about what to do\n"
            f"Action: the action to take, should be one of [{tool_names}]\n"
            "Action Input: the input to the action\n"
            "You must define input according to the following format (no explanations, no markdown):\n"
            "{\n"
            '"tool_name": "Function name",\n'
            '"tool_type": "Type of tool. Only get one of three values ["function", "module", "mcp"]"\n'
            '"arguments": "A dictionary of keyword-arguments to execute tool_name",\n'
            '"module_path": "Path to import the tool"\n'
            "}\n"
            "Observation: the result of the action\n"
            "... (this Thought/Action/Action Input/Observation can repeat N times)\n"
            "Thought: I now know the final answer\n"
            "Final Answer: the final answer to the original input question\n"

            "Begin!\n"

            f"Question: {query}\n"
            "Thought:"
        )    
        return prompt

In [2]:
from langchain_together import ChatTogether 
from vinagent.agent.agent import Agent
from dotenv import load_dotenv
load_dotenv()

llm = ChatTogether(
    model="meta-llama/Llama-3.3-70B-Instruct-Turbo-Free"

)

In [3]:
agent = ReactAgent(
    decription="You are a helpful assistant.",
    skills=["Search on internet"],
    tools=[
        'vinagent/tools/average_dogs.py',
        'vinagent/tools/websearch_tools.py'
    ],
    llm=llm
)

INFO:httpx:HTTP Request: POST https://api.together.xyz/v1/chat/completions "HTTP/1.1 200 OK"
INFO:vinagent.register.tool:Registered weight_of_bulldog:
{'tool_name': 'weight_of_bulldog', 'arguments': {}, 'return': '25', 'docstring': 'The weight of a bulldog', 'dependencies': [], 'module_path': 'vinagent.tools.average_dogs', 'tool_type': 'module', 'tool_call_id': 'tool_97492205-8683-45c4-bf12-e70c8990c27c'}
INFO:vinagent.register.tool:Registered weight_of_husky:
{'tool_name': 'weight_of_husky', 'arguments': {}, 'return': '20', 'docstring': 'The weight of a husky', 'dependencies': [], 'module_path': 'vinagent.tools.average_dogs', 'tool_type': 'module', 'tool_call_id': 'tool_5ef8be21-8aaf-49a1-ac11-e2becd376d21'}
INFO:vinagent.register.tool:Registered average_weight_of_two_dogs:
{'tool_name': 'average_weight_of_two_dogs', 'arguments': {'weight1': 0.0, 'weight2': 0.0}, 'return': '(weight1 + weight2) / 2', 'docstring': 'The average weight of two dogs', 'dependencies': [], 'module_path': 'vin

In [4]:
answer = agent.invoke(query="What is the average weight of two dogs Husky and Bulldog?")
print(answer)

INFO:vinagent.agent.agent:No authentication card provided, skipping authentication
INFO:vinagent.agent.agent:I'am chatting with unknown_user
INFO:vinagent.agent.agent:Tool calling iteration 1/10
INFO:httpx:HTTP Request: POST https://api.together.xyz/v1/chat/completions "HTTP/1.1 200 OK"
INFO:vinagent.agent.agent:Executing tool call: {'tool_name': 'weight_of_husky', 'tool_type': 'module', 'arguments': {}, 'module_path': 'vinagent.tools.average_dogs'}
INFO:vinagent.register.tool:Completed executing module tool weight_of_husky({})
INFO:vinagent.agent.agent:Tool calling iteration 2/10
INFO:httpx:HTTP Request: POST https://api.together.xyz/v1/chat/completions "HTTP/1.1 200 OK"
INFO:vinagent.agent.agent:Executing tool call: {'tool_name': 'weight_of_bulldog', 'tool_type': 'module', 'arguments': {}, 'module_path': 'vinagent.tools.average_dogs'}
INFO:vinagent.register.tool:Completed executing module tool weight_of_bulldog({})
INFO:vinagent.agent.agent:Tool calling iteration 3/10
INFO:httpx:HTTP

content='We have already calculated the average weight of a Husky and a Bulldog, which is 22.5.\n\nThought: I now know the final answer\nFinal Answer: 22.5' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 40, 'prompt_tokens': 3028, 'total_tokens': 3068, 'completion_tokens_details': None, 'prompt_tokens_details': None, 'cached_tokens': 0}, 'model_name': 'meta-llama/Llama-3.3-70B-Instruct-Turbo-Free', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-ed4ba12c-2710-4bc7-a20c-f713ae876f89-0' usage_metadata={'input_tokens': 3028, 'output_tokens': 40, 'total_tokens': 3068, 'input_token_details': {}, 'output_token_details': {}}


In [6]:
print(answer.content)

We have already calculated the average weight of a Husky and a Bulldog, which is 22.5.

Thought: I now know the final answer
Final Answer: 22.5
